In [ ]:
from tflite_runtime.interpreter import Interpreter
import cv2
import numpy as np
import time
def LoadLabelmap(FileName):
    f = open(FileName)
    lines = f.read().splitlines()
    f.close()
    return lines
labelmap = LoadLabelmap('Model/FoClass_Tflite_MobileNetV2_OID/labelmap.txt')

#====讀取模型====
EdgeTpu = False
if (EdgeTpu == True):
    from tflite_runtime.interpreter import load_delegate
    interpreter = Interpreter(model_path=r'Model/FoClass_Tflite_MobileNetV2_OID/detect_edgetpu.tflite',experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
else:
    interpreter = Interpreter(model_path=r'Model/FoClass_Tflite_MobileNetV2_OID/detect_8.tflite')
interpreter.allocate_tensors() #讀取配置模型
Input_Info = interpreter.get_input_details() #取得輸入節點資訊，取得輸入解析度與輸入節點的序號
Output_Info = interpreter.get_output_details() #取得輸出節點資訊
#====讀取圖型====
MyCam = cv2.VideoCapture('ValidateVideo.mp4') #使用影片物件檢測，改成Mcv2.VideoCapture(0)可以使用攝影機
while(MyCam.isOpened()):
    StartTime = time.time()
    ret, cvimg = MyCam.read()
    if (ret == False): #影片讀取結束或是讀取錯誤，所以讀取失敗
        break
    #====處理圖型 成為適合tensorflw格式====
    cvimgRGB = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB) #OpenCV顏色轉換成RGB顏色順序
    Width = Input_Info[0]['shape'][2]
    Height = Input_Info[0]['shape'][1]
    cvimgRGB_Resized = cv2.resize(cvimgRGB, (Width,Height)) 
    Input_data = np.expand_dims(cvimgRGB_Resized, axis=0) #將原本的3維陣列 [寬,高,RGB]  轉成4維度的陣列[第幾張圖片, 寬，高，RGB]
    if (Input_Info[0]['dtype'] == np.float32): #判斷輸入的型態，是否需要將圖型正規劃(將數值由0~255轉換到-1~1)
        Input_data = (np.float32(Input_data) - 128) / 128 #要將輸入資料0-255的顏色資訊轉成 -1 ~ 1 的正規劃數值
    #====使用模型推理====
    interpreter.set_tensor(Input_Info[0]['index'],Input_data)
    interpreter.invoke() #執行模型
    #====取出結果====
    detection_boxes    =interpreter.get_tensor(Output_Info[0]['index'])[0] # 輸出檢測框       [第0張圖片][第幾個框檢測結果][4個座標] 4個座標分別為:0起始座標y , 1起始座標x , 2終點座標y , 3終點座標x
    detection_classes  =interpreter.get_tensor(Output_Info[1]['index'])[0] # 輸出檢測類別     [第0張圖片][第幾個框檢測結果]
    detection_scores   =interpreter.get_tensor(Output_Info[2]['index'])[0] # 輸出檢測信信指數 [第0張圖片][第幾個框檢測結果]
    num_detections     =interpreter.get_tensor(Output_Info[3]['index'])[0] # 輸出檢測數量    [第0張圖片]
    #====繪出結果====
    for i in range(int(num_detections)):
        Score = float(detection_scores[i])
        ClassID = int(detection_classes[i])
        if (Score > 0.5):
            y1 = int(detection_boxes[i][0] * cvimgRGB.shape[0])
            x1 = int(detection_boxes[i][1] * cvimgRGB.shape[1])
            y2 = int(detection_boxes[i][2] * cvimgRGB.shape[0])
            x2 = int(detection_boxes[i][3] * cvimgRGB.shape[1])
            cv2.rectangle(cvimg,(x1,y1),(x2,y2),(0,255,0),thickness=3)
            info = 'Class:{} Score:{}'.format(labelmap[ClassID], round(Score,2))   
            cv2.putText(cvimg, info,(x1+5,y1+30), cv2.FONT_HERSHEY_COMPLEX,1, (255, 255, 0), 2)
    EndTime = time.time()
    Dtime = EndTime - StartTime
    cv2.putText(cvimg, str(int(Dtime*1000)) + 'ms', (0 + 5, 0 + 30), cv2.FONT_HERSHEY_COMPLEX,1, (0, 255, 255), 1)
    cv2.imshow('Press q to exit.',cvimg) 
    if cv2.waitKey(1) == ord('q'):
                break
MyCam.release()
cv2.destroyAllWindows()